# 任务 4：使用您自己的容器执行数据处理

在之前的笔记本中，您使用了 Amazon SageMaker Processing 和 scikit-learn 内置容器执行数据处理。

在此笔记本中，您将设置使用自己的处理容器运行 scikit-learn 脚本所需的环境。

您将创建自己的 Docker 映像，构建处理容器，并使用来自 SageMaker Python SDK 的 **ScriptProcessor** 类在容器中运行 scikit-learn 预处理脚本。

最后，您将验证保存在 Amazon Simple Storage Service (Amazon S3) 中的数据处理结果。

## 任务 4.1：环境设置

在此任务中，您将安装所需的库和依赖项。

您将设置 Amazon S3 存储桶来存储处理作业的输出，还将让执行角色运行 SageMaker 处理作业。

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

## 任务 4.2：创建处理容器

在此任务中，您将使用 Dockerfile 定义并创建 scikit-learn 容器。

### 任务 4.2.1：创建 Dockerfile 

在此任务中，您将创建 Docker 目录，并添加用于创建处理容器的 Dockerfile。因为要创建 scikit-learn 容器，所以要安装 Pandas 和 scikit-learn。

In [ ]:
%mkdir docker

In [ ]:
%%writefile docker/Dockerfile
FROM public.ecr.aws/docker/library/python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

### 任务 4.2.2：构建容器映像

在此任务中，您将使用 Amazon SageMaker Studio 映像构建命令行界面 (CLI) 创建一个自定义容器映像。

通过使用 Amazon SageMaker Studio 映像构建 CLI，您可以直接在 SageMaker Studio 环境中构建兼容 Amazon SageMaker 的 Docker 映像。

安装 SageMaker Studio 映像构建包：

In [ ]:
%pip install sagemaker-studio-image-build

导航到包含 Dockerfile 的目录并运行 sm-docker 构建命令。此命令会自动记录构建输出，并返回 Docker 映像的**映像 URI**。完成此过程大约需要 2 分钟。

In [ ]:
%%sh

cd docker

sm-docker build .

接着，复制**映像 URI** 并将其粘贴到您选择的文本编辑器中。
您将使用此**映像 URI** 来创建一个 **ScriptProcessor** 类。

## 任务 4.3：运行 SageMaker 处理作业

在此任务中，您将使用与前一个笔记本相同的预处理数据集。

In [ ]:
#import-data
shape=pd.read_csv("data/adult_data.csv", header=None)
shape.sample(5)



然后，您将使用 SageMaker ScriptProcessor 类来定义一个处理脚本并将其作为处理作业运行。有关此类的更多信息，请参阅 [SageMaker ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor)。

要创建 ScriptProcessor 类，请配置以下参数：
- **base_job_name**：处理作业名称的前缀
- **command**：要运行的命令，以及任何命令行标志
- **image_uri**：要用于处理作业的 Docker 映像的 URI
- **role**：SageMaker 执行角色
- **instance_count**：要运行处理作业的实例的数量
- **instance_type**：用于处理作业的 Amazon Elastic Compute Cloud (Amazon EC2) 实例的类型

在下面的代码中，将 **REPLACE_IMAGE_URI** 替换为文本编辑器中的 URI。

In [ ]:
#sagemaker-script-processor
from sagemaker.processing import ScriptProcessor

# create a ScriptProcessor
script_processor = ScriptProcessor(
    base_job_name="own-processing-container",
    command=["python3"],
    image_uri="REPLACE_IMAGE_URI",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)


接着，使用 ScriptProcessor.run() 方法将 **sklearn_preprocessing.py** 脚本作为处理作业运行。这与任务 3 中使用的脚本相同，但是您现在是在从基本映像构建的自定义容器上运行该脚本。有关此方法的更多信息，请参阅 [ScriptProcessor.run()](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor.run)。

要运行处理作业，请配置以下参数：
- **code**：预处理脚本的路径 
- **inputs**：预处理脚本的输入数据路径（Amazon S3 输入位置）
- **outputs**：预处理脚本的输出路径（Amazon S3 输出位置）
- **arguments**：预处理脚本的命令行参数（如训练测试分割率）

完成处理作业大约需要 4–5 分钟。

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
script_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "adult_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                         source="/opt/ml/processing/test",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)

## 任务 4.4：验证数据处理结果

通过查看训练和测试输出数据集的前五行，验证您运行的处理作业的输出。

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

### 总结

恭喜！ 您已经成功构建了自己的处理容器，并使用了 SageMaker Processing 来运行处理作业。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。